In [1]:
from pyspark import SparkContext
from pyspark.sql import SQLContext
sc = SparkContext()
sqlCtx = SQLContext(sc)

In [3]:
data_path = 'hdfs://namenode/datasets/github/json/*'

In [2]:
data_path = 'hdfs://namenode/datasets/github/json/contents000000000000.json.gz'

In [3]:
j = sqlCtx.read.json(data_path)
j_rdd = j.toJSON()

In [6]:
def partition(j):
    NUM_BUCKETS = 250
    import json
    j = json.loads(j)
    # ideally, use something like murmurhash instead because hash() implementations vary
    bucket_num = hash(j["id"]) % 250 
    return (bucket_num, json.dumps(j))

In [4]:
def process_line2(j):
    import json
    j = json.loads(j)
    try:
        from lib2to3.refactor import RefactoringTool, get_fixers_from_package
        fixers = get_fixers_from_package('lib2to3.fixes')
        refactoring_tool = RefactoringTool(fixer_names=fixers)
        node3 = refactoring_tool.refactor_string(j["content"], 'script')
        py3_str = str(node3)
        j["content"] = py3_str
        return j
    except:
        return None

In [5]:
j_rdd2 = j_rdd.map(process_line2).filter(lambda x: x)

In [6]:
collected = j_rdd2.take(10)

In [7]:
p = sc.parallelize(collected)

In [8]:
def to_pairs(j):
    return (j["id"], j)

In [10]:
p.map(to_pairs).partitionBy(5).saveAsTextFile("hdfs://namenode/datasets/github/json3/contents", compressionCodecClass="org.apache.hadoop.io.compress.GzipCodec")

In [11]:
step1_data_path = 'hdfs://namenode/datasets/github/json3/contents/part-00000.gz'

In [12]:
j2 = sqlCtx.read.json(step1_data_path)
j2_rdd = j.toJSON()

In [14]:
j2_rdd.first()["binary"]

TypeError: string indices must be integers